# Getting Started with ABLATOR 🚀 and Dashboard

Dashboard is an experimental feature for visualizing, storing and monitoring your experiments remotely. 

## Pre-requisites

You will need a remote storage credentials set-up with a 3rd-party provider such as AWS, GCP or you can even set-up your own SSH server. You can find more about how to set-up your own personal remote storage [HERE](https://github.com/fostiropoulos/rmount)

### fuse3

ABLATOR uses [rmount](https://github.com/fostiropoulos/rmount) in the background to create a Fuse file-system mountpoint that automatically synchronizes the contents

```bash
$ apt-get install fuse3
```

### WARNING

Distributed applications have many moving parts that all have to play nice together for them to work. There are a number of things that can go wrong when trying to synchronize your file system and experiments with a remote storage and are also impossible to mitigate from the ABLATOR end. 

This feature is experimental and you should expect to encounter errors when using it. Reporting errors, issues and sharing your story with us can help us improve our library [HERE](https://github.com/fostiropoulos/ablator/issues)



In [ ]:
!apt-get install fuse3
try:
    import ablator
except ImportError:
    !pip install ablator
    print("Stopping RUNTIME! Please run again.")
    import os

    os.kill(os.getpid(), 9)

### Example of S3 Configuration

```python

remote_config = RemoteConfig(
    s3={
        "provider": "AWS", # or GCP
        "region": "us-east-1",
        "access_key_id": "xxx",
        "secret_access_key": "xxx",
    },
    remote_path=Path("some-bucket"), # s3://some-bucket (you must create it in your account settings)
)
```

SSH configuration requires additional tinkering and is **NOT** recommended for first-time users.


To find out how to set-up secret credentials you can use the official guide for [AWS](https://aws.amazon.com/blogs/security/wheres-my-secret-access-key/) and [GCP](https://cloud.google.com/storage/docs/authentication/managing-hmackeys). 

In [ ]:
from pathlib import Path
import shutil
from torch import nn
import torch
from ablator import (
    ModelConfig,
    ModelWrapper,
    OptimizerConfig,
    TrainConfig,
    configclass,
    Literal,
    ParallelTrainer,
    SearchSpace,
    ParallelConfig,
    RemoteConfig,
)

remote_config = RemoteConfig(
    s3={
        "provider": "AWS",
        "region": "us-east-2",
        "access_key_id": "xxx",
        "secret_access_key": "xxx",
    },
    remote_path=Path("ablator-mock"),  # s3://some-bucket
)

## Simple Experiment

We set-up a simple experiment where we ablate the use of two layer types

#### Layer-A
```python
self.param = nn.Parameter(torch.ones(50, 1))
```
#### Layer-B

```python
self.param = nn.Parameter(torch.randn(200, 1))
```


There are two things to note, our simple experiment modifies the size of the layer i.e. `50` vs `200` and the initialization i.e. `randn` (random) vs `ones`. 

In [ ]:
@configclass
class TrainConfig(TrainConfig):
    dataset: str = "random"
    dataset_size: int


@configclass
class ModelConfig(ModelConfig):
    layer: Literal["layer_a", "layer_b"] = "layer_a"


@configclass
class ParallelConfig(ParallelConfig):
    model_config: ModelConfig
    train_config: TrainConfig


config = ParallelConfig(
    experiment_dir=Path("/tmp/ablator-exp"),
    train_config=TrainConfig(
        batch_size=128,
        epochs=2,
        dataset_size=100,
        optimizer_config=OptimizerConfig(name="sgd", arguments={"lr": 0.1}),
        scheduler_config=None,
    ),
    model_config=ModelConfig(),
    device="cpu",
    search_space={
        "model_config.layer": SearchSpace(categorical_values=["layer_a", "layer_b"])
    },
    total_trials=10,
    remote_config=remote_config,
)


class SimpleModel(nn.Module):
    def __init__(self, config: ModelConfig) -> None:
        super().__init__()
        if config.layer == "layer_a":
            self.param = nn.Parameter(torch.ones(50, 1))
        else:
            self.param = nn.Parameter(torch.randn(200, 1))

    def forward(self, x: torch.Tensor):
        x = self.param * x
        return {"preds": x}, x.sum().abs()


class SimpleWrapper(ModelWrapper):
    def make_dataloader_train(self, run_config: ParallelConfig):
        dl = [torch.rand(100) for i in range(run_config.train_config.dataset_size)]
        return dl

    def make_dataloader_val(self, run_config: ParallelConfig):
        dl = [torch.rand(100) for i in range(run_config.train_config.dataset_size)]
        return dl

## To the stars! 🚀

In [ ]:
if __name__ == "__main__":
    mywrapper = SimpleWrapper(SimpleModel)
    shutil.rmtree(config.experiment_dir, ignore_errors=True)
    with ParallelTrainer(mywrapper, config) as runner:
        runner.launch(".")

## Use Dashboard (even while the experiment is running)

[Dashboard](https://dashboard.ablator.org)

Create an experiment and follow the instructions
![Create an Experiment](assets/create_experiment.png)

![Step 1](assets/step_1.png)

### Compare Trial Configurations

![Configuration Heading](assets/config_bar.png)
![Configuration Comparison](assets/configurations.png)

### Review Logs

![Logs Heading](assets/logs_bar.png)
![Logs](assets/logs.png)


### Review Performance

![Time-Series Heading](assets/timeseries_bar.png)

![Results](assets/timeseries.png)
